# Worksheet 08

Name:  Al Mbaye
UID: U36811669

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.0262447621859865, 5.657627794935047, 5.837855939799306, 5.41535826943198, 3.531255485773168, 4.528801272198257, 5.074887200825845, 4.661152216067535, 3.9763421344554883, 5.4831337073253845]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8, 1 , 10 ).tolist()
print(s2)

[9.990170983203859, 7.860285134550463, 6.886683358012993, 7.082371427737429, 6.107278723547298, 7.864536668394574, 8.227574708396467, 8.881469561929931, 7.170508351645418, 7.5127563884695805]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.4831337073253845, 3.9763421344554883, 7.5127563884695805, 7.170508351645418, 4.661152216067535, 5.074887200825845, 8.881469561929931, 8.227574708396467, 7.864536668394574, 6.107278723547298]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The data originates from a Gaussian Mixture Distribution with two mixture components. The parameters and their known values include:

-there are two distinct species or distributions.
-probability that a data point is drawn from each species is 0.5
-for each species, determining the mean, variance, and probabilities will provide valuable insights into the underlying distribution of the data

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.4831337073253845, 3.9763421344554883, 4.661152216067535, 5.074887200825845, 6.107278723547298]
[7.5127563884695805, 7.170508351645418, 8.881469561929931, 8.227574708396467, 7.864536668394574]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 5.06055879644431,  mean_2 = 7.931369135767194
var_1 = 0.5218897701710409,  var_2 = 0.3498081809521643


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x_i = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_0 | X_i) = P(X_i | S_0)P(S_0) / P(X_i)
    prob_s0_x_i = (pdf_i[0] * prob_s[0]) / prob_x_i
    prob_s0_x.append( prob_s0_x_i )
    # P(S_1 | X_i) = P(X_i | S_1)P(S_1) / P(X_i)
    prob_s1_x_i = (pdf_i[1] * prob_s[1]) / prob_x_i
    prob_s1_x.append( prob_s1_x_i )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_0 = " + str(p[1]))
    print("Probability of coming from S_1 = " + str(p[2]))
    print()


point =  5.4831337073253845
probability of observing that point if it came from cluster 0 =  0.550764546303593
probability of observing that point if it came from cluster 1 =  2.6334861021134202e-11
point =  3.9763421344554883
probability of observing that point if it came from cluster 0 =  0.08833617128003206
probability of observing that point if it came from cluster 1 =  1.9895391767173486e-28
point =  7.5127563884695805
probability of observing that point if it came from cluster 0 =  1.228074907045648e-05
probability of observing that point if it came from cluster 1 =  0.5573262950488165
point =  7.170508351645418
probability of observing that point if it came from cluster 0 =  0.0002157973623166522
probability of observing that point if it came from cluster 1 =  0.10709331262973666
point =  4.661152216067535
probability of observing that point if it came from cluster 0 =  0.5703597027725548
probability of observing that point if it came from cluster 1 =  1.2000741264016791e-19
poi

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)  ]
var = [sum([x[0] * ((x[1] - mean[0])**2) for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * ((x[1] - mean[1])**2) for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 5.061415178922478,  mean_2 = 7.931672056605907
var_1 = 0.5234921902909357,  var_2 = 0.34972497808703557


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
prob_s0_x = []  # P(S_0 | X_i)
prob_s1_x = []  # P(S_1 | X_i)

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    # P(S_0 | X_i) = P(X_i | S_0)P(S_0) / P(X_i)
    prob_s0_x_i = (pdf_i[0] * prob_c[0]) / (pdf_i[0] * prob_c[0] + pdf_i[1] * prob_c[1])
    prob_s0_x.append(prob_s0_x_i)

    # P(S_1 | X_i) = P(X_i | S_1)P(S_1) / P(X_i)
    prob_s1_x_i = (pdf_i[1] * prob_c[1]) / (pdf_i[0] * prob_c[0] + pdf_i[1] * prob_c[1])
    prob_s1_x.append(prob_s1_x_i)

# Print updated posterior probabilities
probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_0 = " + str(p[1]))
    print("Probability of coming from S_1 = " + str(p[2]))
    print()

5.4831337073253845
Probability of coming from S_0 = 0.9999999999530634
Probability of coming from S_1 = 4.6936488488673746e-11

3.9763421344554883
Probability of coming from S_0 = 1.0
Probability of coming from S_1 = 2.1477183740849715e-27

7.5127563884695805
Probability of coming from S_0 = 2.3727488236017437e-05
Probability of coming from S_1 = 0.999976272511764

7.170508351645418
Probability of coming from S_0 = 0.0021288269420479994
Probability of coming from S_1 = 0.997871173057952

4.661152216067535
Probability of coming from S_0 = 1.0
Probability of coming from S_1 = 2.0481396999453567e-19

5.074887200825845
Probability of coming from S_0 = 0.9999999999999952
Probability of coming from S_1 = 4.846040380935625e-15

8.881469561929931
Probability of coming from S_0 = 7.306927132301043e-11
Probability of coming from S_1 = 0.9999999999269308

8.227574708396467
Probability of coming from S_0 = 1.0898008478270854e-08
Probability of coming from S_1 = 0.9999999891019916

7.86453666839457

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [8]:
hard_assignments = []  # List to store hard cluster assignments

for p_s0, p_s1 in zip(prob_s0_x, prob_s1_x):
    if p_s0 > p_s1:
        hard_assignments.append(0)  # Assign to cluster 0
    else:
        hard_assignments.append(1)  # Assign to cluster 1

# Print the hard assignments
for i, assignment in enumerate(hard_assignments):
    print(f"Data point {i} belongs to Cluster {assignment}")

Data point 0 belongs to Cluster 0
Data point 1 belongs to Cluster 0
Data point 2 belongs to Cluster 1
Data point 3 belongs to Cluster 1
Data point 4 belongs to Cluster 0
Data point 5 belongs to Cluster 0
Data point 6 belongs to Cluster 1
Data point 7 belongs to Cluster 1
Data point 8 belongs to Cluster 1
Data point 9 belongs to Cluster 0
